# Initializations

In [1]:
rm(list = ls())
options(error = recover)

library('parallel')
library('ggplot2')
library('reshape')

# __________________________________________________
# Initializations...

setwd('~/EnergyAnalytics/batch/')
source('../utils/aggregate_data.r')

# select one year of data to use
DATA_PATH     = '~/S3L_server/energy-data/pecan_street/usage-processed/'
DATA_OUT_PATH = '~/S3L_server/energy-data/pecan_street/usage-select'
dir.create(DATA_OUT_PATH)
dir.create(paste(DATA_OUT_PATH, "60min/", sep="/"))

In [2]:
PLOTS_PATH    = '~/S3L_server/plots/pecan-street-2'
dir.create(PLOTS_PATH)

Warning message:
In dir.create(PLOTS_PATH): '/Users/adrianalbert/S3L_server/plots/pecan-street-2' already exists

# Load data

In [3]:
# load weather data
weather.hourly = read.csv('~/S3L_server/energy-data/pecan_street/weather/weather_hourly.csv')
weather.15mins = read.csv('~/S3L_server/energy-data/pecan_street/weather/weather_15mins.csv')
weather.hourly$date = as.POSIXct(as.character(weather.hourly$date))
weather.15mins$date = as.POSIXct(as.character(weather.15mins$date))

# load user names
user_names = read.csv('~/S3L_server/energy-data/pecan_street/metadata/user_names_ids.csv')

# __________________________________________________
# Access usage data

# list all data files by year/uid
files.proc  = list.files(DATA_OUT_PATH, full.names = TRUE, recursive = T)
uids.done   = unique(sapply(files.proc, function(x) {
  tmp = strsplit(x, '/')[[1]]
  uid = tmp[length(tmp)]
  uid = strsplit(uid, '\\.')[[1]][1]
  return(uid)
}))
files.input = list.files(DATA_PATH, full.names = TRUE, recursive = T)
uids.input  = unique(sapply(files.input, function(x) {
  tmp = strsplit(x, '/')[[1]]
  uid = tmp[length(tmp)]
  uid = strsplit(uid, '_')[[1]][1]
  return(uid)
}))

# only process those files not processed before
idx.ok   = which(!(uids.input %in% uids.done))
if (length(idx.ok) == 0) stop("All files have already been processed!")

# select files with appropriate resolution 
files_01 = files.input[grep('minute',files.input)]
files_15 = files.input[grep('15mins',files.input)]
files_60 = files.input[grep('hourly', files.input)]

In [4]:
length(files_60)

[1] 1205

#### Concatenate data across multiple years

In [5]:
# ------------------------------------------
# Concatenate data across multiple years
# ------------------------------------------

concatenate_data = function(data_list, dateCol = 'date') {
  
  if (length(data_list) == 1) return(data_list[[1]])
  
  # make sure all end-uses are aligned
  all_cols = unique(unlist(lapply(data_list, names)))
  i = 1
  while (is.null(data_list[[i]]) & i < length(data_list)) i = i + 1; 
  if (i<=length(data_list)) data = data_list[[i]] else return(NULL)
  for (i in 2:length(data_list)) {
    tmp = data_list[[i]]
    if(is.null(tmp)) next
    cur_cols = setdiff(names(data), names(tmp))
    new_cols = setdiff(names(data_list[[i]]), names(data))
    if (length(cur_cols)>0)  {
      tmp[,cur_cols] = NA
    }
    if(length(new_cols)>0) {
      data[,new_cols]= NA;
    }
    data = rbind(data, tmp)
  }
  
  if (!is.null(data)) data[,dateCol] = as.POSIXct(data[,dateCol])
  return(data)
}


In [6]:
if(!require(plyr)){
    install.packages("plyr")}

rm.large<-colwise(function(x){
  return(ifelse(x > 100, NA, x))})

data = list()
for (i in 1:length(uids.input[idx.ok])){ 

  # current user info
  uid = uids.input[idx.ok[i]]  
  user_name = as.character(user_names$name[which(user_names$ID == uid)])
  print(paste('Processing uid ', uid, ' : ', i, '/', length(idx.ok), '\n', sep = ''))

  # get data for current user
  files_60_i = files_60[grep(paste('/', uid, '_hourly.csv', sep=''), files_60)]
  data_list_60 = lapply(files_60_i, function(f) {
    # get current data  
    data = read.csv(f)   
    return(data)
  })
    
  # concatenate data across years
  data_60_sel = concatenate_data(data_list_60)  
  names(data_60_sel)[names(data_60_sel)=="use"] <- "total"
    
  # mark unreasonably high values by NA
  dates = data_60_sel$date
  data_60_sel$date <- NULL
  data_60_sel = rm.large(data_60_sel)
  data_60_sel$date = dates
      
  # skip users that don't have enough data, at least a month of hourly data
  if (nrow(data_60_sel) < 30 * 24) {
    print('Not enough samples (less than one month!) !\n')
    next
    #return(NULL)
  }  
  
  # add in weather (temperature) data 
  data_60_sel = merge(data_60_sel, subset(weather.hourly, select = c('date', 'TemperatureF')), by = 'date')  
  names(data_60_sel)[names(data_60_sel)=="TemperatureF"] <- "Temperature"
  
  # skip users that don't have enough data, at least a month of hourly data
  if (nrow(data_60_sel) < 30 * 24) {
    print('Not enough samples (less than one month!) !\n')
    next
    #return(NULL)
  }  
    
  data[[uid]] = data_60_sel
#   return(data_60_sel)
# })
}

Loading required package: plyr

Attaching package: ‘plyr’

The following objects are masked from ‘package:reshape’:

    rename, round_any



[1] "Processing uid 1103 : 1/555\n"
[1] "Processing uid 1114 : 2/555\n"
[1] "Processing uid 114 : 3/555\n"
[1] "Processing uid 1153 : 4/555\n"
[1] "Processing uid 1169 : 5/555\n"
[1] "Processing uid 1185 : 6/555\n"
[1] "Processing uid 1192 : 7/555\n"
[1] "Processing uid 1283 : 8/555\n"
[1] "Processing uid 1334 : 9/555\n"
[1] "Processing uid 1464 : 10/555\n"
[1] "Processing uid 1508 : 11/555\n"
[1] "Processing uid 1601 : 12/555\n"
[1] "Processing uid 1642 : 13/555\n"
[1] "Processing uid 1697 : 14/555\n"
[1] "Processing uid 171 : 15/555\n"
[1] "Processing uid 1714 : 16/555\n"
[1] "Processing uid 1718 : 17/555\n"
[1] "Processing uid 1782 : 18/555\n"
[1] "Processing uid 1790 : 19/555\n"
[1] "Processing uid 1791 : 20/555\n"
[1] "Processing uid 1792 : 21/555\n"
[1] "Processing uid 1800 : 22/555\n"
[1] "Processing uid 1801 : 23/555\n"
[1] "Processing uid 187 : 24/555\n"
[1] "Processing uid 1953 : 25/555\n"
[1] "Processing uid 2062 : 26/555\n"
[1] "Processing uid 2072 : 27/555\n"
[1] "Processi

In [7]:
user_names[user_names$name=='Nelson',]

,X,ID,name
138,138,6836,Nelson


In [8]:
length(data)

[1] 443

In [9]:
'6836' %in% names(data)

[1] TRUE

In [10]:
dim(data[['6836']])

[1] 16708    20

In [11]:
summary(data[['6836']])

      date                         total              air1       
 Min.   :2012-05-03 17:00:00   Min.   :0.07867   Min.   :0.0000  
 1st Qu.:2012-10-25 11:45:00   1st Qu.:0.15593   1st Qu.:0.0000  
 Median :2013-04-17 13:30:00   Median :0.28875   Median :0.0000  
 Mean   :2013-04-17 12:16:00   Mean   :0.63819   Mean   :0.1373  
 3rd Qu.:2013-10-08 14:15:00   3rd Qu.:0.73035   3rd Qu.:0.1353  
 Max.   :2014-03-31 16:00:00   Max.   :4.61957   Max.   :1.3742  
                               NA's   :211       NA's   :127     
   bathroom1         bedroom1           car1       clotheswasher1  
 Min.   :0.0000   Min.   :0.0000   Min.   :0.000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.000   1st Qu.:0.0000  
 Median :0.0000   Median :0.0000   Median :0.000   Median :0.0000  
 Mean   :0.0012   Mean   :0.0030   Mean   :0.162   Mean   :0.0008  
 3rd Qu.:0.0000   3rd Qu.:0.0015   3rd Qu.:0.000   3rd Qu.:0.0000  
 Max.   :0.0588   Max.   :0.0499   Max.   :3.379   Max.   :0.073

# Study usage across users and appliances by season
It's unclear what appliances are "cooling" and what are "heating". So let's look at what appliances are used primarily in the Summer vs Winter.

In [12]:
cons_season = list()
library("lubridate")
for (uid in names(data)) {
  # current user info
  data_60= data[[uid]]
  user_name = as.character(user_names$name[which(user_names$ID == uid)])
  # skip users that don't have enough data, at least a month of hourly data
  if (nrow(data_60) < 30 * 24) {
    cat('Not enough samples (less than one month!) !\n')
    next
  }
    
  # Compute basic stats about end-uses
  # --------------------------------------------
  
  # prepare data...
  df = cbind(data_60, Hour = hour(data_60$date), Month = month(data_60$date, label=T))
  df$Time = 'Night'
  df$Time[which(df$Hour %in% 5:11)]  = 'Morning'
  df$Time[which(df$Hour %in% 12:18)] = 'Afternoon'
  df$Time[which(df$Hour %in% 19:21)] = 'Evening'
  df$Season = 'Winter'
  df$Season[which(df$Month %in% c("Mar", 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', "Nov"))] = 'Summer'
  #df[is.na(df)] <- 0
    
  # consumption by time of day  
  cur_mu_time = aggregate(. ~ Time + Season, na.action=NULL,
                          FUN = function(x) {if (!is.null(x)) return(mean(x, na.rm=T)) else return(0)},
                          data = df[,-which(names(df) %in% c('date', 'Temperature', 'Hour', 'Month'))])
  cur_mu_time = melt(cur_mu_time, id.vars = c('Time', 'Season'))
  cur_mu_time$ID = uid; cur_mu_time$name = user_name;   
  # consumption by hour of day
  cur_mu_hour = aggregate(. ~ Hour + Season, na.action=NULL,
                          FUN = function(x) {if (!is.null(x)) return(mean(x, na.rm=T)) else return(0)},
                          data = df[,-which(names(df) %in% c('date', 'Temperature', 'Time', 'Month'))])
  cur_mu_hour = melt(cur_mu_hour, id.vars = c('Hour', 'Season'))
  cur_mu_hour$ID = uid; cur_mu_hour$name = user_name;   
  # consumption by season
  cur_mu_season = aggregate(. ~ Season, na.action=NULL,
                            FUN = function(x) {if (!is.null(x)) return(mean(x, na.rm=T)) else return(0)},
                          data = df[,-which(names(df) %in% c('date', 'Temperature', 'Time', 'Month', 'Hour'))])
  cur_mu_season = melt(cur_mu_season, id.vars = c('Season'))
  cur_mu_season$ID = uid; cur_mu_season$name = user_name;   
                          
  cons_season[[uid]] = list(hod=cur_mu_hour, tod=cur_mu_time, seas=cur_mu_season)
}


Attaching package: ‘lubridate’

The following object is masked from ‘package:plyr’:

    here

The following object is masked from ‘package:reshape’:

    stamp



In [13]:
df_seas = do.call("rbind", lapply(cons_season, function(l) l[['seas']]))

In [14]:
str(df_seas)

'data.frame':	9022 obs. of  5 variables:
 $ Season  : chr  "Summer" "Winter" "Summer" "Winter" ...
 $ variable: Factor w/ 70 levels "total","gen",..: 1 1 2 2 1 1 2 2 1 1 ...
 $ value   : num  0.636 0.381 0.671 0.486 1.147 ...
 $ ID      : chr  "1103" "1103" "1103" "1103" ...
 $ name    : chr  "John" "John" "John" "John" ...


In [15]:
avg_tot_seas = aggregate(. ~ variable + Season, FUN = function(x) mean(x, na.rm=T),
                        data = df_seas[,-which(names(df_seas) %in% c('ID', 'name'))])
avg_tot_seas_w = cast(avg_tot_seas, variable~Season, mean)
rownames(avg_tot_seas_w) <- avg_tot_seas_w$variable
avg_tot_seas_w$variable = NULL
avg_tot_seas_w = as.data.frame(avg_tot_seas_w)  
avg_tot_seas_w = avg_tot_seas_w[!rownames(avg_tot_seas_w) %in% c('unknown1', 'unknown4'),] 
head(avg_tot_seas_w)

,Summer,Winter
total,1.181547,0.9388732
gen,0.8265601,0.6180913
air1,0.3036983,0.06499117
bathroom1,0.0144852,0.02105121
bedroom1,0.04172266,0.040295
bedroom2,0.05049719,0.05213969


In [16]:
'total' %in% rownames(avg_tot_seas_w)

[1] TRUE

In [17]:
ratio_season = as.matrix(avg_tot_seas_w['Winter'] / avg_tot_seas_w['Summer'])
ord_idx = order(ratio_season)
ratio_season = ratio_season[ord_idx,]

In [18]:
ratio_season[ratio_season<0.8]

air3       pump1 diningroom2        air1        air2    bedroom4 
 0.05517566  0.08141582  0.15840760  0.21399916  0.28867322  0.48233916 
      drye1  sprinkler1       oven1 winecooler1   disposal1         gen 
 0.67576322  0.69142777  0.70503517  0.72021230  0.74356578  0.74778752 
   freezer1       total 
 0.77800040  0.79461361

In [19]:
ratio_season[ratio_season>1.2]

housefan1 outsidelights_plugs2        lights_plugs4 
            1.214962             1.241982             1.272487 
           venthood1            bathroom2         waterheater1 
            1.284241             1.304632             1.334656 
               oven2       clotheswasher1             bedroom3 
            1.374392             1.375149             1.392217 
           bathroom1                shed1              garage2 
            1.453291             1.685071             1.688922 
        waterheater2             jacuzzi1             unknown3 
            1.898718             2.011621             2.419993 
          poollight1              heater1             bedroom5 
            2.670576             2.954971             3.517813

#### Define categories of usage based on seasonality

In [20]:
# some interesting components
# appliances   = as.character(read.csv('~/energy-data/pecan_street/metadata/appliances.csv')$Appliance)
select.keep  = c('dataid', 'localminute', 'use')
select.AC    = c("air1", "air2", "air3", "diningroom2")
select.HV    = c("heater1", "bedroom3", "unknown3", "bedroom5", "waterheater2", "jacuzzi1")
select.light = c("lights_plugs1", "lights_plugs2", "lights_plugs3", "lights_plugs4", "lights_plugs5", "lights_plugs6",
                 "outsidelights_plugs1", "outsidelights_plugs2")
select.alwOn = c('refridgerator1', 'refridgerator2', 'winecooler1', 'aquarium1', "freezer1")
select.sched = c("pool1", "pool2", 'sprinkler1', "poolpump1", "pump1")
select.total = c('total')
select.dhw   = c('waterheater1', 'waterheater2')
select.user  = c("bathroom1", "bathroom2", "bedroom1", "bedroom2", "bedroom3", "bedroom4", "bedroom5",
                 "clotheswasher1", "clotheswasher_dryg1", "diningroom1", "diningroom2", "dishwasher1",
                 "disposal1", "drye1", "dryg1", "garage1", "garage2", "icemaker1", "jacuzzi1", 
                 "kitchenapp1", "kitchenapp2", "livingroom1", "livingroom2", "heater1",
                 "microwave1", "office1", "oven1", "poollight1",  "range1", "security1", "shed1", "utilityroom1", "venthood1")
select.user  = setdiff(select.user, c(select.AC, select.HV))

select.solar = c('gen')
select.ev    = c('car1')


# Combine end uses by functionality and dump to file

In [115]:
cols = c(total= '#000000', C = '#FE2E2E', H = '#0040FF', nonHVAC = '#088A08', user = '#5F04B4', 
         lights = '#B45F04', always_on = '#01A9DB', scheduled = '#FF00BF', Temperature = '#424242')

In [116]:
# function to aggregate data into broader categories
to_categories = function(homeData, dateCol = 'date') {
  names(homeData) = tolower(names(homeData))
  
  # what kind of data does this user have?
  hv.cols       = intersect(names(homeData), select.HV)
  ac.cols       = intersect(names(homeData), select.AC)
  user.cols     = intersect(names(homeData), select.user)
  alwOn.cols    = intersect(names(homeData), select.alwOn)
  sched.cols    = intersect(names(homeData), select.sched)
  light.cols    = intersect(names(homeData), select.light)
  
  # if no HVAC data, not interesting
  if (length(hv.cols) == 0 && length(ac.cols) == 0) return(NULL)
  if (!(select.total %in% names(homeData))) return(NULL)
  
  # aggregate components
  AC_kwh        = add.columns(homeData, select.AC)
  HV_kwh        = add.columns(homeData, select.HV)
  total_kwh     = add.columns(homeData, select.total)
#   user_kwh      = add.columns(homeData, select.user)
  light_kwh     = add.columns(homeData, select.light)
  sched_kwh     = add.columns(homeData, select.sched)
  alwOn_kwh     = add.columns(homeData, select.alwOn)
#   nonHVAC_kwh   = total_kwh - AC_kwh - HV_kwh - alwOn_kwh - sched_kwh
  user_kwh   = total_kwh - AC_kwh - HV_kwh - alwOn_kwh - sched_kwh - light_kwh
  
  # create new dataset
  data_new = subset(homeData, select = c(dateCol, select.total))
  names(data_new)[names(data_new)==select.total[1]] <- "total"
  if (length(ac.cols)>0) data_new = cbind(data_new, C = AC_kwh)
  if (length(hv.cols)>0) data_new = cbind(data_new, H = HV_kwh)
  if (length(user.cols)>0) data_new = cbind(data_new, user = user_kwh)
  if (length(light.cols)>0) data_new = cbind(data_new, lights = light_kwh)
  if (length(alwOn.cols)>0) data_new = cbind(data_new, always_on = alwOn_kwh)
  if (length(sched.cols)>0) data_new = cbind(data_new, scheduled = sched_kwh)
  #data_new = cbind(data_new, nonHVAC = nonHVAC_kwh)
  
  return(data_new)
}

In [133]:
# plot usage by given factor
plot_comps = function(dat, var = 'Hour', title = 'Components', facet = NULL) {
    
  p = ggplot(dat, aes_string(x = var, y = 'value'))
  p = p + geom_area(aes(fill = variable, color = variable), position = 'stack')    
  if (!is.null(facet)) 
    p = p + facet_wrap(as.formula(paste("~", facet)), ncol=2, scales = 'fixed')
  p = p + scale_color_manual(values = cols) + scale_fill_manual(values = cols)
  p = p + theme_bw() +
    theme(panel.grid.major  = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          strip.text.x     = element_text(size=18),
          axis.title.x     = element_text(size = 18),
          axis.text.y      = element_text(size=18), 
          axis.text.x      = element_text(size=18),
          axis.title.y     = element_text(size=18),
          axis.title.x     = element_text(size=18),
          plot.title       = element_text(size=20),            
          legend.text      = element_text(size=18),        
          legend.title     = element_blank(),    
          legend.position  = c(0.18, 0.45),
          legend.direction="horizontal",
          axis.ticks = element_blank() ) + 
    theme(plot.title=element_text(family="Times", face="bold", size=20)) + 
    ggtitle(title)  + xlab(var) + ylab('Avg. kWh')
  
  return(p)
}

In [134]:
# function to plot ground truth components
f_plot_user = function(homeData0, main = NULL) {

  #homeData$nonHVAC = NULL
  maxUsage = Inf; noTries = 0
  while (is.infinite(maxUsage) & noTries < 5){
      idx = sample(1:(nrow(homeData0)-400),1)
      TT  = 250 
      noTries = noTries + 1
      homeData = homeData0[idx:(idx+TT),]
      maxUsage = max(homeData$total, na.rm=T)
  }
  if (is.infinite(maxUsage)) return(NULL)
  homeData$date = as.POSIXct(homeData$date)
  # plot different usages
  plot(homeData$date,homeData$total,type='l',col=cols['total'], lwd=2,
       main=main, ylab='kWh', xlab=NA, ylim=c(0,1.1*maxUsage), xaxt='n',
       cex.main=1.5, cex.axis=1.5, cex.lab=1.5)
  for (var in setdiff(names(homeData), c('total', 'date'))) {
    points(homeData$date,homeData[,var],type='l',col=cols[var], lwd=1.8)
  }
  
  # if plotting temperature, adjust values to plot on same axis
  if ('Temperature' %in% names(homeData))
    points(homeData$date, homeData$TemperatureF/100*maxUsage,type='l',col=cols['Temperature'], cex=1.5)
  
  d = homeData$date
  dts = seq(d[1],d[length(d)],by=3600*24) # one per day from one per minute
  axis(1, dts, format(dts, "%a, %m/%d"), cex.axis=1.5)
  grid(nx=NA,ny=NULL)
  abline(v=dts,col="black",lty=3)  
  vars_sel = setdiff(names(homeData), 'date')
  legend("top", lty=1,cex=1.5,lwd=3,  x.intersp=0.0, legend=vars_sel, col=cols[vars_sel],horiz=T)

}

In [150]:
for (uid in names(data)) {
  # current user info
  data_60= data[[uid]]
  user_name = as.character(user_names$name[which(user_names$ID == uid)])
  # skip users that don't have enough data, at least a month of hourly data
  if (nrow(data_60) < 30 * 24) {
    cat('Not enough samples (less than one month!) !\n')
    next
  }
    
  # aggregate end-uses
  cur_data = to_categories(data_60)
  if (is.null(cur_data)) {
    cat(paste('No HVAC data for user', uid, '!\n'))
    next
  } else {
    cat(paste("Plotting for", uid, "\n"))
  }
    
  # produce aggregate plot
  cur_data = cbind(cur_data, Hour = hour(cur_data$date), Month = month(cur_data$date, label=T))
  cur_data$Season = 'Winter'
  cur_data$Season[which(cur_data$Month %in% c("Mar", 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', "Nov"))] = 'Summer'
  cur_data$Month <- NULL
  cur_mu_hour = aggregate(. ~ Hour + Season, na.action=NULL,
                          FUN = function(x) {if (!is.null(x)) return(mean(x, na.rm=T)) else return(0)},
                          data = cur_data[,!(names(cur_data) %in% c('date', 'Temperature', 'Time', 'Month'))])
  cur_mu_hour = cur_mu_hour[intersect(names(cur_mu_hour),c('Hour', 'Season','C', 'H', 'user'))]
  cur_mu_hour = melt(cur_mu_hour, id.vars = c('Hour', 'Season'))
  cur_mu_hour$ID = uid; cur_mu_hour$name = user_name;   
    
  # plot end-uses
  dir.create(file.path(PLOTS_PATH, uid), recursive = TRUE)
  p.hr = plot_comps(cur_mu_hour, 
                    var = 'Hour', facet = 'Season',
                    title = paste('Avg. Hourly Usage for', user_name))
  pdf(file=paste(paste(PLOTS_PATH, uid, sep = '/'), paste(uid,'_components.pdf',sep=''), sep='/'),width=9,height=3)
  print(p.hr)
  dev.off()
                                             
  # plot example time series
  pdf(file=paste(paste(PLOTS_PATH, uid, sep = '/'), paste(uid,'_example_timeseries.pdf',sep=''), sep='/'),width=8.5,height=4) 
  f_plot_user(cur_data[,!(names(cur_data) %in% c("Hour", "Season"))], main=paste("Example time series:", user_name))
  dev.off()
    
  # save data to file
  cur_data['Temperature'] = data_60$Temperature
  write.csv(cur_data[,!(names(cur_data) %in% c("Season", "Hour"))], 
            file = paste(DATA_OUT_PATH, paste('60min/', uid, '.csv', sep = ''), sep = '/'), row.names = F)  
}

No HVAC data for user 1103 !
No HVAC data for user 1114 !
Plotting for 114 


Warning message:
In dir.create(file.path(PLOTS_PATH, uid), recursive = TRUE): '/Users/adrianalbert/S3L_server/plots/pecan-street-2/114' already exists

No HVAC data for user 1153 !
Plotting for 1169 


Warning message:
In dir.create(file.path(PLOTS_PATH, uid), recursive = TRUE): '/Users/adrianalbert/S3L_server/plots/pecan-street-2/1169' already exists

Plotting for 1185 


...

Warning message:
In dir.create(file.path(PLOTS_PATH, uid), recursive = TRUE): '/Users/adrianalbert/S3L_server/plots/pecan-street-2/9931' already exists

Plotting for 9938 


Warning message:
In dir.create(file.path(PLOTS_PATH, uid), recursive = TRUE): '/Users/adrianalbert/S3L_server/plots/pecan-street-2/9938' already exists

Plotting for 9983 


Warning message:
In dir.create(file.path(PLOTS_PATH, uid), recursive = TRUE): '/Users/adrianalbert/S3L_server/plots/pecan-street-2/9983' already exists

In [106]:
summary(cur_data)

      date                         total              C           
 Min.   :2012-07-23 23:00:00   Min.   :0.1269   Min.   :0.000000  
 1st Qu.:2012-12-02 15:00:00   1st Qu.:0.3774   1st Qu.:0.000000  
 Median :2013-04-13 02:00:00   Median :0.7294   Median :0.001433  
 Mean   :2013-04-21 09:47:19   Mean   :0.9938   Mean   :0.301131  
 3rd Qu.:2013-08-22 12:00:00   3rd Qu.:1.3462   3rd Qu.:0.434817  
 Max.   :2014-03-31 16:00:00   Max.   :5.5156   Max.   :2.163800  
                               NA's   :130                        
      user             nonHVAC             Hour         Season         
 Min.   :0.000000   Min.   :-2.1625   Min.   : 0.0   Length:12617      
 1st Qu.:0.000933   1st Qu.: 0.2420   1st Qu.: 6.0   Class :character  
 Median :0.027750   Median : 0.4645   Median :11.0   Mode  :character  
 Mean   :0.087284   Mean   : 0.6824   Mean   :11.5                     
 3rd Qu.:0.052117   3rd Qu.: 0.8815   3rd Qu.:17.0                     
 Max.   :3.232167   Max.   : 5.2

In [145]:
head(cur_data)

,date,total,C,user,Hour,Season
1,2012-05-03 17:00:00,1.6285,0.6975,0.931,17,Summer
2,2012-05-03 18:00:00,NA,0.642,-0.642,18,Summer
3,2012-05-03 19:00:00,3.7605,0.4185,3.342,19,Summer
4,2012-05-03 20:00:00,1.5435,0.4935,1.05,20,Summer
5,2012-05-03 21:00:00,0.641,0.256,0.385,21,Summer
6,2012-05-03 22:00:00,0.4475,0.1955,0.252,22,Summer


In [146]:
head(data[['6836']])

,date,total,air1,bathroom1,bedroom1,car1,clotheswasher1,dishwasher1,disposal1,drye1,furnace1,gen,kitchenapp1,kitchenapp2,livingroom1,microwave1,refrigerator1,unknown1,unknown2,Temperature
1,2012-05-03 17:00:00,1.6285,0.6975,NA,NA,NA,NA,NA,NA,0,NA,0.0925,NA,NA,NA,NA,NA,0,0,90.71783
2,2012-05-03 18:00:00,NA,0.642,NA,NA,NA,NA,NA,NA,0,NA,-0.006,NA,NA,NA,NA,NA,0,0.0015,88.91567
3,2012-05-03 19:00:00,3.7605,0.4185,NA,NA,NA,NA,NA,NA,0,NA,-0.0025,NA,NA,NA,NA,NA,0,0,87.43533
4,2012-05-03 20:00:00,1.5435,0.4935,NA,NA,NA,NA,NA,NA,0,NA,-0.003,NA,NA,NA,NA,NA,0,0,85.22833
5,2012-05-03 21:00:00,0.641,0.256,NA,NA,NA,NA,NA,NA,0,NA,-0.002,NA,NA,NA,NA,NA,0,0,82.51533
6,2012-05-03 22:00:00,0.4475,0.1955,NA,NA,NA,NA,NA,NA,0,NA,-0.002,NA,NA,NA,NA,NA,0,0,80.3595


In [147]:
cur_data = to_categories(data_60)

In [148]:
head(cur_data)

,date,total,C,user
1,2012-05-03 17:00:00,1.6285,0.6975,0.931
2,2012-05-03 18:00:00,NA,0.642,-0.642
3,2012-05-03 19:00:00,3.7605,0.4185,3.342
4,2012-05-03 20:00:00,1.5435,0.4935,1.05
5,2012-05-03 21:00:00,0.641,0.256,0.385
6,2012-05-03 22:00:00,0.4475,0.1955,0.252
